In [1]:
import os
os.environ['KB_AUTH_TOKEN'] = open('/tmp/kb_auth_token.txt').read().strip()

In [2]:
from biokbase import data_api
from biokbase.data_api import display

In [3]:
b = data_api.browse(654)

In [4]:
genomes = [item.object for item in b
    if item.type.startswith('KBaseGenomesCondensedPrototypeV2.GenomeAnnotation')]

In [15]:
g0 = genomes[0]

In [12]:
class Classification(display.TemplateMixin):
    """Taxonomic classification.

    Attributes:
      taxon (TaxonAPI): base taxon
      name (str): Scientific name
      children (list of TaxonAPI): List of TaxonAPI objects
      parents (list of Taxon
    """
    template = '''{% for name in classification %}
    <span style="margin-left: {{ loop.index0 * 10 }}px">
    <span style="font-size: 50%">&gt;</span>&nbsp;{{ name }}
    </span><br>{% endfor %}'''

    def __init__(self, obj):
        """Create from a taxon.

        Args:
          obj: TaxonAPI object or object with `get_taxon`.
        """
        display.TemplateMixin.__init__(self)
        self.taxon = obj.get_taxon() if hasattr(obj, 'get_taxon') else obj
        self.name = self.taxon.get_scientific_name()
        self.children = self.taxon.get_children() or []
        tx, self.parents = self.taxon, []
        while tx:
            tx = tx.get_parent()
            if tx:
                self.parents.insert(tx.get_scientific_name(), 0)
        self.classification = self.parents + [self.name] + [
            child.get_scientific_name() for child in self.children]

    def _repr_html_(self):
        return self.render(classification=self.classification)


In [16]:
Classification(g0)